In [73]:
import json
import os
from unskript import nbparams
from unskript.secrets import ENV_MODE, ENV_MODE_AWS
from unskript.fwk.workflow import Task, Workflow

env = {"ENV_MODE":"ENV_MODE_UNSKRIPT_HOSTED","TENANT_ID":"3c475b94-d99e-477e-bceb-7edf74500898","PROXY_ID":"af32736b-fc75-4bdc-8785-8183cdeba8e1","TENANT_URL":"https://us.app.unskript.io","AWS_REGION":"us-west-2"}
secret_store_cfg = {"SECRET_STORE_TYPE":"SECRET_STORE_TYPE_AWS","AWS_SECRET_PREFIX":"","AWS_REGION":"us-west-2"}
os.environ["UNSKRIPT_REDIS_HOST"] = "redis-master.redis.svc.cluster.local"
os.environ["UNSKRIPT_TOKEN"] = "5c4a5754-0600-11ec-9a03-0242ac130003"
os.environ["TENANT_URL"] = env["TENANT_URL"]
paramDict = {"user_name": "FoobarAdmin"}
paramDict.update(env)
paramDict.update(secret_store_cfg)
paramsJson = json.dumps(paramDict)
nbParamsObj = nbparams.NBParams(paramsJson)
user_name = nbParamsObj.get('user_name')

w = Workflow(env, secret_store_cfg, None, global_vars=globals())


<img src="https://unskript.com/assets/favicon.png" alt="unSkript.com" width="100" height="100"/> 
<h1> unSkript Runbooks </h1>
<div class="alert alert-block alert-success">
    <b> This runbook demonstrates How to delete IAM user using unSkript legos.</b>
</div>

<br>

<center><h2>Delete IAM User</h2></center>

# Steps Overview
1) Delete login profile for given user.
2) Get list of all Access policies for the user.
3) Remove each policy found.
4) .Delete IAM user by passing User name. 
5) Check the caller identity of the current user.
6) Send success IAM user deletion message to the slack channel

Here we will Delete the ogin Profile for an IAM User. This lego takes UserName as input, and deletes the login profile for the user.

In [74]:
##
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import Dict
from botocore.exceptions import ClientError
import pprint

from beartype import beartype
@beartype
def aws_delete_user_login_profile_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_delete_user_login_profile(handle, UserName: str) -> Dict:
    """aws_create_user_login_profile Create login profile for IAM User.

        :type UserName: string
        :param UserName: Name of new IAM User.

 

        :rtype: Dict with the Profile Creation status info.
    """

    ec2Client = handle.client("iam")
    result = {}
    try:
        response = ec2Client.delete_login_profile(
            UserName=UserName)

        result = response
    except ClientError as error:
        if error.response['Error']['Code'] == 'EntityAlreadyDeleted':
            result = error.response
        else:
            result = error.response

    return result


task = Task(Workflow())
task.configure(credentialsJson='''{
    "credential_name": "doug aws",
    "credential_type": "CONNECTOR_TYPE_AWS",
    "credential_id": "2abcb384-0732-4257-94bd-6f4d3374890e"
}''')

task.configure(inputParamsJson='''{
    "UserName": "user_name"
    }''')



(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_delete_user_login_profile, lego_printer=aws_delete_user_login_profile_printer, hdl=hdl, args=args)

This user may have multiple policies that we have to delete before we delete the user

In [78]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from typing import List, Dict
from pydantic import BaseModel, Field
from botocore.exceptions import ClientError
import pprint


from beartype import beartype
@beartype
def aws_list_attached_user_policies_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_list_attached_user_policies(handle, user_name: str) -> List:
    """aws_list_attached_user_policies returns the list of policies attached to the user.

        :type handle: object
        :param handle: Object returned from task.validate(...).

        :type user_name: string
        :param user_name: IAM user whose policies need to fetched.

        :rtype: List with with the attched policy names.
    """
    result = []
    ec2Client = handle.client('iam')
    try:
        response = ec2Client.list_attached_user_policies(UserName=user_name)
        for i in response["AttachedPolicies"]:
            result.append(i['PolicyName'])

    except ClientError as error:
        result.append(error.response)

    return result


task = Task(Workflow())
task.configure(credentialsJson='''{
    "credential_name": "doug aws",
    "credential_type": "CONNECTOR_TYPE_AWS",
    "credential_id": "2abcb384-0732-4257-94bd-6f4d3374890e"
}''')
task.configure(inputParamsJson='''{
    "user_name": "user_name"
    }''')
task.configure(outputName="policies")
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_list_attached_user_policies, lego_printer=aws_list_attached_user_policies_printer, hdl=hdl, args=args)

In [77]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from typing import List, Dict
from pydantic import BaseModel, Field
from botocore.exceptions import ClientError
import pprint


from beartype import beartype

def aws_detach_iam_policy(handle, user_name: str, policy_name) -> Dict:
    """aws_attache_iam_policy used to provide user permissions.
        :type handle: object
        :param handle: Object returned from task.validate(...).
        :type user_name: string
        :param user_name: Dictionary of credentials info.
        :type policy_name: string
        :param policy_name: Policy name to apply the permissions to the user.
        :rtype: Dict with User policy information.
    """
    for policy in policy_name:
    
        result = {}
        iamResource = handle.resource('iam')
        try:
            user = iamResource.User(user_name)
            response = user.detach_policy(
                PolicyArn='arn:aws:iam::aws:policy/'+policy
                )
            result = response
        except ClientError as error:
            result = error.response

    return result


def unskript_default_printer(output):
    if isinstance(output, (list, tuple)):
        for item in output:
            print(f'item: {item}')
    elif isinstance(output, dict):
        for item in output.items():
            print(f'item: {item}')
    else:
        print(f'Output for {task.name}')
        print(output)

task = Task(Workflow())
task.configure(credentialsJson='''{
    "credential_name": "doug aws",
    "credential_type": "CONNECTOR_TYPE_AWS",
    "credential_id": "2abcb384-0732-4257-94bd-6f4d3374890e"
}''')
task.configure(inputParamsJson='''{
    "policy_name": "policies",
    "user_name": "user_name"
    }''')

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_detach_iam_policy, lego_printer=unskript_default_printer, hdl=hdl, args=args)

Here we will use unSkript Delete IAM User Lego. This lego takes user_name as input. This inputs is used to delete the IAM user from AWS.

In [79]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from typing import List, Dict
from pydantic import BaseModel, Field
from botocore.exceptions import ClientError
import pprint

from beartype import beartype
@beartype
def aws_delete_iam_user_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_delete_iam_user(handle, user_name: str) -> Dict:
    """aws_create_iam_user Creates new IAM User.

        :type user_name: string
        :param user_name: Name of new IAM User.


        :rtype: Dict with the stopped instances state info.
    """

    ec2Client = handle.client("iam")
    result = {}
    try:
        response = ec2Client.delete_user(
            UserName=user_name)
        result = response
    except ClientError as error:
        if error.response['Error']['Code'] == 'EntityAlreadyExists':
            result = error.response
        else:
            result = error.response

    return result


task = Task(Workflow())
task.configure(credentialsJson='''{
    "credential_name": "doug aws",
    "credential_type": "CONNECTOR_TYPE_AWS",
    "credential_id": "2abcb384-0732-4257-94bd-6f4d3374890e"
}''')

task.configure(inputParamsJson='''{
    "user_name": "user_name"
    }''')
task.configure(outputName="UserInfo")
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_delete_iam_user, lego_printer=aws_delete_iam_user_printer, hdl=hdl, args=args)

Here we will use unSkript STS Get Caller Identity Lego. This lego did't take any inputs. Shows the the caller identity for the current user.

In [80]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from typing import List, Dict
from pydantic import BaseModel, Field
import pprint


from beartype import beartype
@beartype
def aws_get_caller_identity_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_get_caller_identity(handle) -> Dict:
    ec2Client = handle.client('sts')
    response = ec2Client.get_caller_identity()

    return response


task = Task(Workflow())
task.configure(outputName="caller")

task.configure(credentialsJson='''{
    "credential_name": "doug aws",
    "credential_type": "CONNECTOR_TYPE_AWS",
    "credential_id": "2abcb384-0732-4257-94bd-6f4d3374890e"
}''')

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_get_caller_identity, lego_printer=aws_get_caller_identity_printer, hdl=hdl, args=args)

Here we will use unSkript Post Slack Message Lego. This lego takes channel name and message as input. This inputs is used to to post the message when new IAM user created.

In [81]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##

import pprint

from pydantic import BaseModel, Field
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError

pp = pprint.PrettyPrinter(indent=2)


from beartype import beartype
@beartype
def slack_post_message_printer(output):
    if output is not None:
        pprint.pprint(output)
    else:
        return


@beartype
def slack_post_message(
        handle: WebClient,
        channel: str,
        message: str) -> str:

    try:
        response = handle.chat_postMessage(
            channel=channel,
            text=message)
        return f"Successfuly Sent Message on Channel: #{channel}"
    except SlackApiError as e:
        pp.pprint(
            f"Failed sending message to slack channel {channel}, Error: {e.response['error']}")
        if e.response['error'] == 'channel_not_found':
            raise Exception('Channel Not Found')
        elif e.response['error'] == 'duplicate_channel_not_found':
            raise Exception('Channel associated with the message_id not valid')
        elif e.response['error'] == 'not_in_channel':
            raise Exception('Cannot post message to channel user is not in')
        elif e.response['error'] == 'is_archived':
            raise Exception('Channel has been archived')
        elif e.response['error'] == 'msg_too_long':
            raise Exception('Message text is too long')
        elif e.response['error'] == 'no_text':
            raise Exception('Message text was not provided')
        elif e.response['error'] == 'restricted_action':
            raise Exception('Workspace preference prevents user from posting')
        elif e.response['error'] == 'restricted_action_read_only_channel':
            raise Exception('Cannot Post message, read-only channel')
        elif e.response['error'] == 'team_access_not_granted':
            raise Exception('The token used is not granted access to the workspace')
        elif e.response['error'] == 'not_authed':
            raise Exception('No Authtnecition token provided')
        elif e.response['error'] == 'invalid_auth':
            raise Exception('Some aspect of Authentication cannot be validated. Request denied')
        elif e.response['error'] == 'access_denied':
            raise Exception('Access to a resource specified in the request denied')
        elif e.response['error'] == 'account_inactive':
            raise Exception('Authentication token is for a deleted user')
        elif e.response['error'] == 'token_revoked':
            raise Exception('Authentication token for a deleted user has been revoked')
        elif e.response['error'] == 'no_permission':
            raise Exception('The workspace toekn used does not have necessary permission to send message')
        elif e.response['error'] == 'ratelimited':
            raise Exception('The request has been ratelimited. Retry sending message later')
        elif e.response['error'] == 'service_unavailable':
            raise Exception('The service is temporarily unavailable')
        elif e.response['error'] == 'fatal_error':
            raise Exception('The server encountered catostrophic error while sending message')
        elif e.response['error'] == 'internal_error':
            raise Exception('The server could not complete operation, likely due to transietn issue')
        elif e.response['error'] == 'request_timeout':
            raise Exception('Sending message error via POST: either message was missing or truncated')
        else:
            raise Exception(f'Failed Sending Message to slack channel {channel} Error: {e.response["error"]}')

    except Exception as e:
        print("\n\n")
        pp.pprint(
            f"Failed sending message to slack channel {channel}, Error: {e.__str__()}")
        return f"Unable to send message on {channel}"


task = Task(Workflow())
task.configure(credentialsJson='''{
    "credential_name": "doug_slack",
    "credential_type": "CONNECTOR_TYPE_SLACK",
    "credential_id": "49e309a8-3ed5-49d6-b437-1bbd8b95a1a2"
}''')

task.configure(inputParamsJson='''{
    "channel": "\\"devrel_doug_test1\\"",
    "message": "f'IAM user {user_name} deleted by {caller[\\"Arn\\"]}'"
    }''')


(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(slack_post_message, lego_printer=slack_post_message_printer, hdl=hdl, args=args)

In this Runbook, we demonstrated the use of unSkript's AWS and slack to perform AWS create new IAM user and login profile for the same user and also show the caller identity of the current user and post the message on the slack channel. To view the full platform capabilities of unSkript please visit https://unskript.com